In [2]:
using DataFrames
using Recommender: Movielens1M, load_dataset, ratio_split, ItemkNN, fit!, predict, evaluate, PrecisionAtK, make_u2i_dataset

In [3]:
ml1m = Movielens1M()
download(ml1m)
rating, user, movie = load_dataset(ml1m);

In [4]:
df = DataFrame(rating)
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,1,1193,5,978300760
2,1,661,3,978302109
3,1,914,3,978301968
4,1,3408,4,978300275
5,1,2355,5,978824291
6,1,1197,3,978302268
7,1,1287,5,978302039
8,1,2804,5,978300719
9,1,594,4,978302268


In [5]:
df = df[df.rating.>=4, :]
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,1,1193,5,978300760
2,1,3408,4,978300275
3,1,2355,5,978824291
4,1,1287,5,978302039
5,1,2804,5,978300719
6,1,594,4,978302268
7,1,919,4,978301368
8,1,595,5,978824268
9,1,938,4,978301752


In [6]:
df[!, :rating] .= 1
first(df, 10)

,userid,movieid,rating,timestamp
,Int64,Int64,Int64,Int64
1,1,1193,1,978300760
2,1,3408,1,978300275
3,1,2355,1,978824291
4,1,1287,1,978302039
5,1,2804,1,978300719
6,1,594,1,978302268
7,1,919,1,978301368
8,1,595,1,978824268
9,1,938,1,978301752


In [7]:
df = ratio_split(df, train_ratio = 0.6, valid_ratio=0.2);

In [8]:
df_train = df[df.data_split.==:train, :]
df_valid = df[df.data_split.==:valid, :]
df_test = df[df.data_split.==:test, :]
df_train_valid = df[df.data_split.!=:test, :]
first(df_train, 10)

,userid,movieid,rating,timestamp,data_split
,Int64,Int64,Int64,Int64,Symbol
1,1,1193,1,978300760,train
2,1,1287,1,978302039,train
3,1,2804,1,978300719,train
4,1,594,1,978302268,train
5,1,919,1,978301368,train
6,1,595,1,978824268,train
7,1,938,1,978301752,train
8,1,2398,1,978302281,train
9,1,1035,1,978301753,train


In [9]:
nrow(df_train), nrow(df_valid), nrow(df_test)

(345168, 115056, 115057)

In [10]:
metric = PrecisionAtK(10)

PrecisionAtK(10)

In [11]:
using TreeParzen

In [12]:
space = Dict(
    :topk=>HP.QuantUniform(:topk, 10., 500., 1.),
    :shrink=>HP.LogUniform(:shrink, log(1e-3), log(1e3)),
    :weighting=>HP.Choice(:weighting, 
        [
            Dict(:weighting=>:dummy, :weighting_at_inference=>false),
            Dict(:weighting=>:tfidf, :weighting_at_inference=>false),
            Dict(:weighting=>:bm25, :weighting_at_inference=>HP.Choice(:weighting_at_inference, [true, false]))
        ]
    ),
    :normalize=>HP.Choice(:normalize, [true, false])
)

Dict{Symbol, TreeParzen.Types.AbstractDelayed} with 4 entries:
  :weighting => Choice(Param(:weighting, RandIndex(3)), Dict{Symbol, Any}[Dict(…
  :topk      => QuantUniform(:topk, QuantUniform(10.0, 500.0, 1.0))
  :normalize => Choice(Param(:normalize, RandIndex(2)), Bool[1, 0])
  :shrink    => LogUniform(:shrink, LogUniform(-6.90776, 6.90776))

In [13]:
function invert_output(params)
    k = convert(Int, params[:topk])
    model = ItemkNN(k, params[:shrink],params[:weighting][:weighting],params[:weighting][:weighting_at_inference],params[:normalize])
    result = evaluate(model, df_train, df_valid, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating, drop_history=false)
    @show params, result
    return -result
end

invert_output (generic function with 1 method)

In [14]:
best = fmin(invert_output, space, 10, logging_interval=-1)

(params, result) = (Dict{Symbol, Any}(:weighting => Dict{Symbol, Any}(:weighting => :tfidf, :weighting_at_inference => false), :topk => 292.0, :normalize => true, :shrink => 84.63887548570325), 0.1119819518716625)


In [ ]:
best_model = ItemkNN(convert(Int, best[:topk]), best[:shrink],best[:weighting][:weighting],best[:weighting][:weighting_at_inference],best[:normalize])
evaluate(best_model, df_train_valid, df_test, metric, 10, col_user=:userid, col_item=:movieid, col_rating=:rating)

LoadError: UndefVarError: best not defined